In [1]:
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
from pynwb import NWBFile, NWBHDF5IO
from pynwb.ecephys import ElectricalSeries, LFP

In [2]:
nwbfile = NWBFile(
    session_description="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
    identifier="TODO",
    session_start_time=datetime(2019, 3, 27),
    lab="Johannes Sarnthein Lab",
    institution="University of Zurich",
)

/home/hhh/.cache/pypoetry/virtualenvs/usz-neuro-conversion-6s2qedde-py3.11/lib/python3.11/site-packages/pynwb/file.py:1113: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [3]:
device = nwbfile.create_device(
    name="NicoletOne EEG", description="TODO", manufacturer="Natus Medical Incorporated"
)
nwbfile.add_electrode_column(name="label", description="label of electrode")

group_count = 19
channels_per_group = 1
electrode_counter = 0

for group in range(group_count):
    # create an electrode group for this group
    electrode_group = nwbfile.create_electrode_group(
        name="group{}".format(group),
        description="electrode group for group {}".format(group),
        device=device,
        location="PZ",
        position=[0, 0, 0],
    )
    # add electrodes to the electrode table
    for channel in range(channels_per_group):
        nwbfile.add_electrode(
            group=electrode_group,
            label="group{}channel{}".format(group, channel),
            location="PZ",
            reference="Averaged mastoid channels",
        )
        electrode_counter += 1
print(nwbfile.electrodes.to_dataframe())

   location                                              group group_name  \
id                                                                          
0        PZ  group0 pynwb.ecephys.ElectrodeGroup at 0x13982...     group0   
1        PZ  group1 pynwb.ecephys.ElectrodeGroup at 0x13982...     group1   
2        PZ  group2 pynwb.ecephys.ElectrodeGroup at 0x13982...     group2   
3        PZ  group3 pynwb.ecephys.ElectrodeGroup at 0x13982...     group3   
4        PZ  group4 pynwb.ecephys.ElectrodeGroup at 0x13982...     group4   
5        PZ  group5 pynwb.ecephys.ElectrodeGroup at 0x13982...     group5   
6        PZ  group6 pynwb.ecephys.ElectrodeGroup at 0x13982...     group6   
7        PZ  group7 pynwb.ecephys.ElectrodeGroup at 0x13982...     group7   
8        PZ  group8 pynwb.ecephys.ElectrodeGroup at 0x13982...     group8   
9        PZ  group9 pynwb.ecephys.ElectrodeGroup at 0x13982...     group9   
10       PZ  group10 pynwb.ecephys.ElectrodeGroup at 0x1398...    group10   

In [8]:
all_table_region = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="all electrodes",
)
# From EEG
raw_data = np.random.randn(50, electrode_counter)
raw_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=raw_data,
    electrodes=all_table_region,
    starting_time=0.0,
    rate=256.0,  # in Hz
)
nwbfile.add_acquisition(raw_electrical_series)
# TODO: stimuli

In [10]:
# From iEEG
lfp_data = np.random.randn(50, electrode_counter)
lfp_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=lfp_data,
    electrodes=all_table_region,
    starting_time=0.0,
    rate=200.0,
)
lfp = LFP(electrical_series=lfp_electrical_series)
module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
module.add(lfp)

ValueError: 'EEG' already exists in NWBFile 'root'

In [ ]:
nwbfile.add_unit_column(name="quality", description="sorting quality")

poisson_lambda = 20
firing_rate = 20
n_units = 10
for n_units_per_shank in range(n_units):
    n_spikes = np.random.poisson(lam=poisson_lambda)
    spike_times = np.round(
        np.cumsum(np.random.exponential(1 / firing_rate, n_spikes)), 5
    )
    nwbfile.add_unit(
        spike_times=spike_times, quality="good", waveform_mean=[1.0, 2.0, 3.0, 4.0, 5.0]
    )

nwbfile.units.to_dataframe()

In [ ]:
# TODO: Add waveforms

In [ ]:
with NWBHDF5IO("ecephys_tutorial.nwb", "w") as io:
    io.write(nwbfile)